# ¿How well does a metric learning approach do?

In [1]:
import torch
from functools import partial
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt


# reproducibility
seed = 1993
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_dir = "./data"

## Load and preprocess data

In [2]:
train_dataset_gpu = {}
eval_dataset_gpu = {}

# dataset
train = torchvision.datasets.CIFAR100(root=data_dir, download=True, transform=transforms.ToTensor())
eval = torchvision.datasets.CIFAR100(root=data_dir, train=False, transform=transforms.ToTensor())

# move dataset to gpu
train_dataset_gpu_loader = torch.utils.data.DataLoader(train, batch_size=len(train), drop_last=True,
                                            shuffle=True, num_workers=2, persistent_workers=False)
eval_dataset_gpu_loader = torch.utils.data.DataLoader(eval, batch_size=len(eval), drop_last=True,
                                            shuffle=False, num_workers=1, persistent_workers=False)
train_dataset_gpu['images'], train_dataset_gpu['targets'] = [item.to(device="cuda", non_blocking=True) for item in next(iter(train_dataset_gpu_loader))]
eval_dataset_gpu['images'],  eval_dataset_gpu['targets']  = [item.to(device="cuda", non_blocking=True) for item in next(iter(eval_dataset_gpu_loader)) ]

# normalize images
train_cifar_std, train_cifar_mean = torch.std_mean(train_dataset_gpu['images'], dim=(0, 2, 3)) 
print(f"Mean: {[f'{x:.4f}' for x in train_cifar_mean.tolist()]}")
print(f"Std: {[f'{x:.4f}' for x in train_cifar_std.tolist()]}")
def batch_normalize_images(input_images, mean, std):
    return (input_images - mean.view(1, -1, 1, 1)) / std.view(1, -1, 1, 1)
batch_normalize_images = partial(batch_normalize_images, mean=train_cifar_mean, std=train_cifar_std)
train_dataset_gpu['images'] = batch_normalize_images(train_dataset_gpu['images'])
eval_dataset_gpu['images']  = batch_normalize_images(eval_dataset_gpu['images'])

data = {
        'train': train_dataset_gpu,
        'eval': eval_dataset_gpu
    }

# pad images for later random cropping
pad_amount = 4
data['train']['images'] = F.pad(data['train']['images'], (pad_amount,)*4, 'reflect')

Files already downloaded and verified
Mean: ['0.5071', '0.4865', '0.4409']
Std: ['0.2673', '0.2564', '0.2762']


### Metric based data loading

In [3]:
from batch_transforms import batch_crop, batch_flip_lr

@torch.no_grad()
def get_batches(data_dict, key, batchsize, paired=True, indices=range(100)):
    # select subset of class indices 
    indices = torch.tensor(indices, device=device)
    images, targets = data_dict[key]["images"], data_dict[key]["targets"] 
    samples = torch.isin(targets, indices)
    images, targets = images[samples], targets[samples]
    
    assert len(images) == len(targets)

    num_epoch_examples = len(images)
    shuffled = torch.randperm(num_epoch_examples, device=device)
    crop_size = 32

    # shuffle the dataset
    images = images[shuffled]
    targets = targets[shuffled]

    # transforms
    if key == 'train':
        images = batch_crop(images, crop_size)
        images = batch_flip_lr(images)

    if paired: 
        # as we are going to pair up the images, we need the size of the dataset to be even
        if len(images) % 2 != 0:
            images = images[:-1]
            targets = targets[:-1]
        # pair up the dataset targets = targets.reshape(num_epoch_examples // 2, 2)
        binary_targets = torch.eq(targets[:,0], targets[:,1])
        # we need that roughly 50% of the pairs are positive and negative
        while binary_targets.float().mean() < 0.5:
            # unpair the target and binary_targets
            targets = targets.reshape(num_epoch_examples)  
            binary_targets = torch.stack([binary_targets, binary_targets], 1).reshape(num_epoch_examples)
            # get negative elements from negative pairs
            neg = binary_targets == False
            # permute them hoping some of them turn into positive pairs
            perm = torch.randperm(len(binary_targets[neg]))
            images[neg] = images[neg][perm]
            targets[neg] = targets[neg][perm]
            # re-pair the targets
            targets = targets.reshape(num_epoch_examples // 2, 2)
            binary_targets = torch.eq(targets[:,0], targets[:,1])
        ## TODO: ensuring 50% distribtution takes much longer than before 
        ## without 50% -> 30 ms
        ## with 50% -> 1 seg
        
        images = images.reshape(num_epoch_examples // 2, 2, 3, images.shape[-1], images.shape[-2])
        num_epoch_examples = len(images)

    for idx in range(num_epoch_examples // batchsize):
        if paired:
            yield images[idx*batchsize: (idx+1)*batchsize, 0], images[idx*batchsize: (idx+1)*batchsize, 1],  binary_targets[idx*batchsize: (idx+1)*batchsize].int()
        else: 
            yield images[idx*batchsize: (idx+1)*batchsize], targets[idx*batchsize: (idx+1)*batchsize]

#### ¿How do I make sure that the implementation is correct?

- ¿Are there any duplicated or overlapping pairs?
    I think there are not since the pairs are selected by reshaping the original 1d-tensor of targets in a matrix with 2 rows, so if there were not any duplicates in the original tensor, then there must not be any duplicates in the pairs. 

In [ ]:
%%time 

for x1, x2, y in get_batches(data, "train", 128):
    pass

In [ ]:
print(x1.shape)
print(x2.shape)

In [ ]:
print(y)

In [ ]:
idx = 0

In [ ]:
from torchvision.transforms.functional import to_pil_image
idx_to_class = {i:c for c,i in train.class_to_idx.items()}

print(idx)
display(to_pil_image(x1[idx]))
display(to_pil_image(x2[idx]))

print(y[idx])
# print([idx_to_class[i] for i in y[idx].tolist()])
idx += 1

In [ ]:
%%time 

for x, y in old_get_batches(data, "train", 128):
    y

In [ ]:
from torchvision.transforms.functional import to_pil_image

n = len(data["train"]["images"])

images_pairs = data["train"]["images"].reshape(n // 2, 2, 3, 40, 40)
labels_pairs = data["train"]["targets"].reshape(n // 2, 2)

# show first and second image
display(to_pil_image(data["train"]["images"][0]))
print(data["train"]["targets"][0])
display(to_pil_image(data["train"]["images"][1]))
print(data["train"]["targets"][1])

# show first pair
display(to_pil_image(images_pairs[0][0]))
display(to_pil_image(images_pairs[0][1]))
print(labels_pairs[0])

## Implement siamese network

### Model definition

In [4]:
x1, x2, y = next(get_batches(data, "train", 128))

In [5]:
print(x1.shape)
print(x2.shape)
print(y.shape)

torch.Size([128, 3, 32, 32])
torch.Size([128, 3, 32, 32])
torch.Size([128])


In [4]:
from torch import nn
from torchvision.models import resnet18

class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = resnet18()
        self.ndim = self.conv.fc.in_features
        self.conv = nn.Sequential(*list(resnet18().children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(self.ndim, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.Linear(256, 2),
        )
     
    def forward_once(self, x):
        x = self.conv(x)         # (bs, ndim, 1, 1)
        x = torch.flatten(x, 1)  # (bs, ndim)
        x = self.fc(x)           # (bs, 2)
        return x 

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)  # (bs, 2)
        output2 = self.forward_once(input2)  # (bs, 2)
        return output1, output2


In [5]:
net = SiameseNetwork()
net.to(device);

In [9]:
out1, out2 = net(x1, x2)
print(out1.shape, out2.shape)

torch.Size([128, 2]) torch.Size([128, 2])


### Classification Mechanism

f() -> (x, y)

1. Calculate mean embedding of each class in the dataset.
2. x -> d(x, mu_i) -> min d(x, mu_i) 1-NN.

## Train model

### Loss function

In [10]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidean distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim=True)
      loss_contrastive = torch.mean((1-label) * euclidean_distance**2 + (label) * torch.clamp(self.margin - euclidean_distance, min=0.0)**2)


      return loss_contrastive    

In [11]:
criterion = ContrastiveLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.0005)

### Train and eval methods

In [ ]:
from batch_transforms import get_batches

def train(optimizer, task):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    indices = range(increment * task, increment * (task + 1))
    for batch_idx, (inputs1, inputs2, targets) in enumerate(get_batches(data, "train", batch_size, indices=indices)):
        optimizer.zero_grad()
        outputs1, outputs2 = net(inputs1, inputs2)
        loss = criterion(outputs1, outputs2, targets)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    return train_loss/(batch_idx + 1)

def eval(from_task, to_task, emb):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    indices = range(from_task * increment, (to_task+1) * increment)
    with torch.no_grad():
        for batch_idx, (inputs1, inputs2, targets) in enumerate(get_batches(data, "eval", batch_size, indices=indices)):
            outputs1, outputs2 = net(inputs1, inputs2)
            loss = criterion(outputs1, outputs2, targets)
            test_loss += loss.item()
        return test_loss/(batch_idx + 1)

def mean_embeddings(net, task):
    mean_embds = torch.tensor(2, (task+1)*increment)
    for idx in range((task+1)*increment):
        x, _ = next(get_batches(data, "train", 500, paired=False, indices=range(idx)))
        print(x.shape)  # (500, 3, 40, 40)
        emb = net(x) # (500, 2)
        mean_emb = emb.mean(0)
        mean_embds[idx] = mean_emb
    return mean_embds

### Incremental training

In [ ]:
def incremental_train(task, to_task)
    # select optimize and scheduler
    if task == 0:
        epochs = init_epochs
        optimizer = optim.SGD(net.parameters(), momentum=0.9, lr=init_lr, weight_decay=init_weight_decay)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    else: 
        epochs = rest_epochs
        optimizer = optim.SGD(net.parameters(), momentum=0.9, lr=rest_lr, weight_decay=rest_weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=rest_milestones, gamma=rest_lr_decay)

    pbar = tqdm(range(epochs), unit="epoch")
    for epoch in pbar:
        tloss, tacc = train(optimizer, task)
        eloss, eacc = eval(from_task=0, to_task=to_task)
        scheduler.step()
        pbar.set_postfix({"Train Loss": tloss, "Train Acc": tacc, "Eval Loss": eloss, "Eval Acc": eacc})

    return eloss, eacc

## Review results

### Accuracy and loss

### Plot the embedding with t-sne